In [3]:
# === World Database on Protected Areas Merging Norway and Denmark ===
import geopandas as gpd
import pandas as pd
import glob
import os

# === CONFIG ===
root_dir = r"C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK"
output_excel = os.path.join(root_dir, "merged_name_coordinates.xlsx")

# === STEP 1: Recursively find all .shp files in nested folders ===
norway_shps = glob.glob(os.path.join(root_dir, "Norway", "**", "*.shp"), recursive=True)
denmark_shps = glob.glob(os.path.join(root_dir, "Denmark", "**", "*.shp"), recursive=True)

shapefile_paths = norway_shps + denmark_shps
print(f"Found {len(shapefile_paths)} shapefiles to merge.")

# === STEP 2: Load and merge ===
gdfs = []
for path in shapefile_paths:
    print(f"Loading: {path}")
    gdf = gpd.read_file(path)

    # Ensure CRS is WGS84
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    gdfs.append(gdf)

merged = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")
print("All shapefiles merged.")

# === STEP 3: Add centroid lat/lon for export ===
merged["longitude"] = merged.geometry.centroid.x
merged["latitude"] = merged.geometry.centroid.y

# === STEP 4: Export to Excel with selected columns ===
# Check for a valid column name
possible_name_cols = ["NAME", "NAME_EN", "WDPAID", "DESIG_ENG", "ORIG_NAME"]
name_col = next((col for col in possible_name_cols if col in merged.columns), None)

if name_col is None:
    raise ValueError("No suitable 'name' column found. Please check the attribute names.")

# Select only name + coordinates
export_df = merged[[name_col, "latitude", "longitude"]].rename(columns={name_col: "name"})

# Save to Excel
export_df.to_excel(output_excel, index=False)

print(f"Excel exported with columns: name, latitude, longitude → {output_excel}")


Found 9 shapefiles to merge.
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Norway\WDPA_WDOECM_Apr2025_Public_NOR_shp_0\WDPA_WDOECM_Apr2025_Public_NOR_shp-points.shp
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Norway\WDPA_WDOECM_Apr2025_Public_NOR_shp_0\WDPA_WDOECM_Apr2025_Public_NOR_shp-polygons.shp
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Norway\WDPA_WDOECM_Apr2025_Public_NOR_shp_1\WDPA_WDOECM_Apr2025_Public_NOR_shp-points.shp
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Norway\WDPA_WDOECM_Apr2025_Public_NOR_shp_1\WDPA_WDOECM_Apr2025_Public_NOR_shp-polygons.shp
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Norway\WDPA_WDOECM_Apr2025_Public_NOR_shp_2\WDPA_WDOECM_Apr2025_Public_NOR_shp-points.shp
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Norway\WDPA_WDOECM_Apr2025_Public_NOR_shp_2\WDPA_WDOECM_Apr2025_Public_NOR_shp-polygons.shp
Loading: C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\Denmark\WDPA_WDOECM_Apr2025_Public_DNK_shp_0\WDPA_WDOE

C:\Users\sakib\AppData\Local\Temp\ipykernel_23596\2677377980.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged["longitude"] = merged.geometry.centroid.x
C:\Users\sakib\AppData\Local\Temp\ipykernel_23596\2677377980.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged["latitude"] = merged.geometry.centroid.y


Excel exported with columns: name, latitude, longitude → C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\merged_name_coordinates.xlsx


In [5]:
# === Final MCDA with Environmental Impact Cosideration ===
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# === Step 1: Load MPA Excel File ===
mpa_df = pd.read_excel(r"C:\GISDataManipulation\WDPA_SHP_ALL_NOR_DNK\merged_name_coordinates.xlsx")  # update path

# Ensure column names match your file
mpa_df = mpa_df.rename(columns={"latitude": "lat", "longitude": "lon", "Name": "mpa_name"})

# Drop rows without coordinates
mpa_df = mpa_df.dropna(subset=["lat", "lon"])

# Create geometry
mpa_df["geometry"] = mpa_df.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
gdf_mpa = gpd.GeoDataFrame(mpa_df, geometry="geometry", crs="EPSG:4326")

# === Step 2: Buffer MPAs (e.g., 5 km) ===
buffer_km = 5
buffer_deg = buffer_km / 111  # Rough conversion: 1° ≈ 111 km

gdf_mpa["geometry"] = gdf_mpa.geometry.buffer(buffer_deg)
mpa_union = gdf_mpa.unary_union  # merge all MPA buffers into one

# === Step 3: Load MCDA Results ===
mcda = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\final_mcda_scores.txt")  # update path
mcda = mcda.dropna(subset=["Latitude", "Longitude"])

# Convert to GeoDataFrame
mcda["geometry"] = mcda.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
gdf_mcda = gpd.GeoDataFrame(mcda, geometry="geometry", crs="EPSG:4326")

# === Step 4: Check if MCDA points fall within buffered MPAs ===
gdf_mcda["in_or_near_mpa"] = gdf_mcda.geometry.within(mpa_union)

# === Option A: Filter out those points ===
mcda_clean = gdf_mcda[~gdf_mcda["in_or_near_mpa"]]
mcda_clean.to_csv(r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\mcda_outside_mpa_buffer.txt", index=False)

# === Option B: Apply penalty to final_score ===
gdf_mcda["env_impact_score"] = gdf_mcda["in_or_near_mpa"].apply(lambda x: 0 if x else 9)
gdf_mcda["final_score_with_env"] = (
    0.3 * gdf_mcda["solar_score"] +
    0.25 * gdf_mcda["wind_score"] +
    0.2 * gdf_mcda["land_score"] +
    0.15 * gdf_mcda["grid_score"] +
    0.1 * gdf_mcda["env_impact_score"]
).round(2)

# Save updated version
gdf_mcda.to_csv(r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\mcda_with_environmental_filter.txt", index=False)

print("Done! MCDA points processed with MPA buffer zone.")


C:\Users\sakib\AppData\Local\Temp\ipykernel_23596\2364959364.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_mpa["geometry"] = gdf_mpa.geometry.buffer(buffer_deg)
C:\Users\sakib\AppData\Local\Temp\ipykernel_23596\2364959364.py:24: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mpa_union = gdf_mpa.unary_union  # merge all MPA buffers into one


Done! MCDA points processed with MPA buffer zone.


In [6]:
# === Final MCDA with Environmental Impact Cosideration Maps Out ===
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# === Step 1: Load the MCDA results with environmental filtering ===
df = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\mcda_with_environmental_filter.txt")  # Update with your path

# === Step 2: Filter points with final_score_with_env > 7 ===
optimal = df[df["final_score_with_env"] > 7.5].copy()
print(f"Found {len(optimal)} optimal locations with score > 7.5")

# === Step 3: Save filtered data to a new file ===
optimal.to_csv(r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\optimal_mcda_locations.txt", index=False)

# === Step 4: Create a Folium Map ===
# Set a rough center for Norway/Denmark
m = folium.Map(location=[60.0, 10.0], zoom_start=5, tiles="CartoDB positron")

# Add a marker cluster for better visualization
marker_cluster = MarkerCluster().add_to(m)

# === Step 5: Add markers to the map ===
for _, row in optimal.iterrows():
    popup_content = (
        f"<b>Final Score (Env):</b> {row['final_score_with_env']}<br>"
        f"<b>Solar:</b> {row['solar_score']}<br>"
        f"<b>Wind:</b> {row['wind_score']}<br>"
        f"<b>Land:</b> {row['land_score']}<br>"
        f"<b>Grid:</b> {row['grid_score']}"
    )
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, max_width=250),
        icon=folium.Icon(color="green", icon="check")
    ).add_to(marker_cluster)

# === Step 6: Save the map to an HTML file ===
m.save(r"C:\GISDataManipulation\MCDA Files\MCDAwEnvironment\optimal_mcda_map.html")
print("Interactive map saved to 'optimal_mcda_map.html'")


Found 15 optimal locations with score > 7.5
Interactive map saved to 'optimal_mcda_map.html'
